In [21]:
from collections import defaultdict
import numpy as np
import math 

In [22]:
def parseData(fname):
  for l in open(fname):
    yield eval(l)

In [23]:
print("Reading data...")
data_user_items = list(parseData("./australian_users_items.json"))
print("Done.")

Reading data...
Done.


In [24]:
print("Reading the second dataset...")
data_user_reviews = list(parseData("./australian_user_reviews.json"))
print("Done for the second.")

Reading the second dataset...
Done for the second.


In [8]:
#############Preprocess the data and put them into dictionary############

In [25]:
#Since there are users that have more than 1 data entry, we first want to integrate
data_integ = defaultdict(list)
#the key is the user, the list[0] is the items list, list[1] is the items_count list
for d in data_user_items: 
    user = d['user_id']
    items = d['items']
    count = d['items_count']
    if len(data_integ[user]) == 0: 
        data_integ[user].append([])
        data_integ[user][0].append(items)
        data_integ[user].append([])
        data_integ[user][1].append(count)
    else: 
        data_integ[user][0].append(items)
        data_integ[user][1].append(count)

In [27]:
print(len(data_integ.keys()))

87626


In [31]:
userIdList = []
for key in data_integ.keys(): 
    userIdList.append(key)

In [33]:
#######Random shuffle the data#####
np.random.shuffle(userIdList)
data_train_userItem = defaultdict(list)
data_validation_userItem = defaultdict(list)
data_test_userItem = defaultdict(list)
index_count = 0 
for key in userIdList: 
    if index_count < 65000: 
        data_train_userItem[key] = data_integ[key]
    elif index_count >= 65000 and index_count < 80000: 
        data_validation_userItem[key] = data_integ[key]
    else: 
        data_test_userItem[key] = data_integ[key]

In [40]:
###########Process to add the item ids into the dictionary###########
def getAllAttributes(data_userItems): 
    user_itemsId = defaultdict(set)
    itemsId_user = defaultdict(set)
    user_items_forever = defaultdict(defaultdict)
    id_names = dict()
    
    user_count = defaultdict(int)
    
    for user in data_userItems.keys(): 
        items_id_list = data_userItems[user][0]
        items_count_list = data_userItems[user][1]
        
        item_id = set() 
        
        for i in items_id_list: 
            for each in i: 
                #add the id 
                item_id.add(each['item_id'])
                itemsId_user[each['item_id']].add(user)
                
                user_items_forever[user][each['item_id']] = each['playtime_forever']
                
                #corresponde to the name for the id 
                id_names[each['item_id']] = each['item_name']
                
        user_itemsId[user] = item_id
        c = 0
        for i in items_count_list: 
            c += i 
        user_count[user] = c 
    return user_itemsId, itemsId_user, user_items_forever, id_names, user_count 

In [41]:
user_itemsId, itemsId_user, user_items_forever, id_names, user_count  = getAllAttributes(data_train_userItem)

In [44]:
#########Feature 1: the most popular items#############3
countUsers = [(len(itemsId_user[key]), key) for key in itemsId_user]
countUsers.sort() 
countUsers.reverse() 

In [45]:
def mostKItems(k, items): 
    topK = [x[1] for x in items[0,k]]
    return topK

In [36]:
#########Feature 2:Calculate item similarity based by purchased Users############
def item_similarity(userItems): 
    N=dict()
    C=defaultdict(defaultdict)
    W=defaultdict(defaultdict)
    for key in userItems: 
        #the items which are purchased by the user
        for i in userItems[key]: 
            if i not in N.keys(): 
                N[i] = 0 
            N[i] += 1 
            for j in userItems[key]: 
                if i == j: 
                    continue
                if j not in C[i].keys(): 
                    C[i][j] = 0 
                C[i][j] += 1 
    print("Finish the first part ")
    for i, related_item in C.items(): 
        for j, cij in related_item.items(): 
            W[i][j] = cij/math.sqrt(N[i]*N[j])
    return W

In [37]:
W = item_similarity(user_itemsId) 

Finish the first part 


In [39]:
#########Feature 3: calculate user cosine similarity based on forever time#############

In [40]:
def norm(user): 
    norm_v = 0 
    for i in user_items_id[user]:
        norm_v += user_items_forever[user][i]
    return norm_v

In [41]:
def allNorms(): 
    N = defaultdict(int)
    for u in user_items.keys():
        N[u] = norm(u)
    return N 

In [42]:
N = allNorms() 

In [38]:
def cosine(items_id_user, user_items_forever, N): 
    C = defaultdict(defaultdict)
    for key in items_id_user: 
        for u1 in items_id_user[key]:
            for u2 in items_id_user[key]: 
                if u1 == u2: 
                    continue 
                if u2 not in C[u1].keys(): 
                    C[u1][u2] = 0
                C[u1][u2] += user_items_forever[u1][key] * user_items_forever[u2][key]

    for key in C:
        for each in C[key]: 
            C[key][each] = C[key][each] / (N[key]*N[each])**0.5
    return C

In [ ]:
C = cosine(itemsId_user, user_items_forever, N)